## Connect to workspace using config file

Save the config file in the same directory as the notebook and run the following code to connect to the workspace.



![](2023-04-10-22-21-36.png)

In [2]:
# using saved workspace from config file
from azureml.core import Workspace

ws = Workspace.from_config()


In [ ]:
#specify workspace details

from azureml.core import Workspace

ws = Workspace.get(name='aml-workspace',
                   subscription_id='1234567-abcde-890-fgh...',
                   resource_group='aml-resources')

In [3]:
#compute targets defined in workspace

for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(compute.name, ":", compute.type)

[SDK documentation](https://aka.ms/AA70zeq)

## Azure ML CLI Extension

authenticate using the following command

```bash
az login
```

compute targets in a workspace (g = resource group, w = workspace)

```bash
az ml computetarget list -g 'aml-resources' -w 'aml-workspace'
```

### experiment run

```python
from azureml.core import Experiment

# create an experiment variable
experiment = Experiment(workspace = ws, name = "my-experiment")

# start the experiment
run = experiment.start_logging()

# experiment code goes here

# end the experiment
run.complete()
```

### Logging metrics

```python
from azureml.core import Experiment
import pandas as pd

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = 'my-experiment')

# Start logging data from the experiment
run = experiment.start_logging()

# load the dataset and count the rows
data = pd.read_csv('data.csv')
row_count = (len(data))

# Log the row count
run.log('observations', row_count)

# Complete the experiment
run.complete()
```

In [ ]:
# view metrics for a run
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
import json

# Get logged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

In [ ]:
# upload files to outputs folder
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

In [ ]:
# Retrieve list of output files
import json

files = run.get_file_names()
print(json.dumps(files, indent=2))

In [ ]:
from azureml.core import Run
import pandas as pd
import matplotlib.pyplot as plt
import os

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
data = pd.read_csv('data.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)

# Save a sample of the data
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# Complete the run
run.complete()

In [ ]:
from azureml.core import Experiment, ScriptRunConfig

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='experiment.py') 

# submit the experiment
experiment = Experiment(workspace = ws, name = 'my-experiment')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)